### Google Map API
Get Latitude and longitude for each location

In [1]:
# please put all the package used here
from collections import Counter
import pandas as pd
import csv
import pprint
import numpy as np
import warnings
import requests
import json
warnings.filterwarnings('ignore')
import psycopg2
import psycopg2.extras
import time
import ast # convert string to dictionary

In [6]:
data_CarSharingPods_dic = list(csv.DictReader(open('new_CarSharingPods.csv')))
data_CarSharingPods = pd.read_csv('new_CarSharingPods.csv')

data_BusinessStats_dic = list(csv.DictReader(open('new_BusinessStats.csv')))
data_BusinessStats = pd.read_csv('new_BusinessStats.csv')

data_CensusStats_dic = list(csv.DictReader(open('CensusStats.csv')))
data_CensusStats = pd.read_csv('CensusStats.csv')

data_StatisticalAreas_dic = list(csv.DictReader(open('StatisticalAreas.csv')))
data_StatisticalAreas = pd.read_csv('StatisticalAreas.csv')

data_Neighbourhoods_dic = list(csv.DictReader(open('new_Neighbourhoods.csv')))
data_Neighbourhoods = pd.read_csv('new_Neighbourhoods.csv')

# retrieve the source code of the webpage
page_html = requests.get("https://en.wikipedia.org/wiki/List_of_Sydney_Trains_railway_stations").text
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
#print(page_html)

In [7]:
def location_boundry(name1):
    try:
        name1 = name1.strip().replace('\n','')
        name1 = name1 +" - Sydney"
        my_params= {'address': name1,'language':'en'}  
        response = requests.get(base_url, params = my_params)
        results  = response.json()['results']
        nhood_bounds = results[0]['geometry']['viewport']
        name1 = name1.replace(" - Sydney","")
        print(name1)
        #print(results)
        x_geo    = results[0]['geometry']['location']
        if name1 in GPS.keys() or name1 in Location.keys():
            print("duplication !!! 🐳🐳🐳")
            return
        else:
            GPS[name1] = [x_geo['lng'],x_geo['lat']]
            Location[name1] = [json.dumps(nhood_bounds, indent=4)]
        
        print("{} GPS : {} {}".format(name1, x_geo['lng'], x_geo['lat']))
        print("Boundary: ", json.dumps(nhood_bounds, indent=4))
        print('🌀-----------------------------------------------------------------🌀')
    except IndexError as e:
        print(name1)
        time.sleep( 1 )  # 过于频繁 抓取失败的话，就休息一秒继续抓
        location_boundry(name1) #recursive
        print('😂-----------------------------------------------------------------😂')

### Add parent area to Neighbourhood

In [8]:
data_Neighbourhoods.head()

,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,GPS_lng,GPS_lat,Location_northeast_lat,Location_northeast_lng,Location_southwest_lat,Location_southwest_lng
0,102011028,Avoca Beach - Copacabana,643.8,7590.0,2325,738.0,151.435000,-33.490000,-33.479474,151.447116,-33.493322,151.422514
1,102011029,Box Head - MacMasters Beach,3208.6,10986.0,3847,907.0,151.411448,-33.496717,-33.481446,151.429207,-33.513661,151.393475
2,102011030,Calga - Kulnura,76795.1,4841.0,1575,1102.0,151.218000,-33.432000,-33.375486,151.249674,-33.449607,151.200898
3,102011031,Erina - Green Point,3379.3,14237.0,4450,1666.0,150.795867,-33.629882,-33.628554,150.797339,-33.631252,150.794641
4,102011032,Gosford - Springfield,1691.2,19385.0,6373,2126.0,151.341667,-33.426667,-33.408444,151.351788,-33.439373,151.329445


In [9]:
data_StatisticalAreas.head()

,area_id,area_name,parent_area_id
0,1,New South Wales,0
1,10,Greater Sydney,1
2,11,Rest of NSW,1
3,2,Victoria,0
4,20,Greater Melbourne,2


In [10]:
name = []
for i in data_StatisticalAreas['area_name']:
    name.append(i)

In [13]:
GPS = {}  # nested list
Location = {}
for name in data_StatisticalAreas["area_name"]:
    location_boundry(name)

New South Wales
New South Wales GPS : 151.2092955 -33.8688197
Boundary:  {
    "northeast": {
        "lat": -33.5781409,
        "lng": 151.3430209
    },
    "southwest": {
        "lat": -34.118347,
        "lng": 150.5209286
    }
}
🌀-----------------------------------------------------------------🌀
Greater Sydney
Greater Sydney GPS : 151.2092955 -33.8688197
Boundary:  {
    "northeast": {
        "lat": -33.8561088,
        "lng": 151.222951
    },
    "southwest": {
        "lat": -33.8797035,
        "lng": 151.1970329
    }
}
🌀-----------------------------------------------------------------🌀
Rest of NSW
Rest of NSW GPS : 151.2092955 -33.8688197
Boundary:  {
    "northeast": {
        "lat": -33.5781409,
        "lng": 151.3430209
    },
    "southwest": {
        "lat": -34.118347,
        "lng": 150.5209286
    }
}
🌀-----------------------------------------------------------------🌀
Victoria
Victoria GPS : 151.2040035 -33.8697767
Boundary:  {
    "northeast": {
        "lat": 

Gosford
Gosford GPS : 151.3178844 -33.4235481
Boundary:  {
    "northeast": {
        "lat": -33.42219911970849,
        "lng": 151.3192333802915
    },
    "southwest": {
        "lat": -33.42489708029149,
        "lng": 151.3165354197085
    }
}
🌀-----------------------------------------------------------------🌀
Avoca Beach - Copacabana
Avoca Beach - Copacabana GPS : 151.435 -33.49
Boundary:  {
    "northeast": {
        "lat": -33.4794739,
        "lng": 151.4471164
    },
    "southwest": {
        "lat": -33.4933219,
        "lng": 151.4225143
    }
}
🌀-----------------------------------------------------------------🌀
Box Head - MacMasters Beach
Box Head - MacMasters Beach GPS : 151.4114482 -33.4967169
Boundary:  {
    "northeast": {
        "lat": -33.4814458,
        "lng": 151.4292066
    },
    "southwest": {
        "lat": -33.5136605,
        "lng": 151.3934751
    }
}
🌀-----------------------------------------------------------------🌀
Calga - Kulnura
Calga - Kulnura GPS : 1

Jilliby - Yarramalong
Jilliby - Yarramalong GPS : 151.285 -33.223
Boundary:  {
    "northeast": {
        "lat": -33.1840415,
        "lng": 151.300915
    },
    "southwest": {
        "lat": -33.2386539,
        "lng": 151.2082689
    }
}
🌀-----------------------------------------------------------------🌀
😂-----------------------------------------------------------------😂
Lake Munmorah - Mannering Park
Lake Munmorah - Mannering Park GPS : 151.5398251 -33.1508781
Boundary:  {
    "northeast": {
        "lat": -33.1495291197085,
        "lng": 151.5411740802915
    },
    "southwest": {
        "lat": -33.1522270802915,
        "lng": 151.5384761197085
    }
}
🌀-----------------------------------------------------------------🌀
Ourimbah - Fountaindale - Sydney
Ourimbah - Fountaindale
Ourimbah - Fountaindale GPS : 151.393 -33.34
Boundary:  {
    "northeast": {
        "lat": -33.329752,
        "lng": 151.4195877
    },
    "southwest": {
        "lat": -33.3660104,
        "lng": 151.38

Baulkham Hills (West) - Bella Vista
Baulkham Hills (West) - Bella Vista GPS : 150.955 -33.742
Boundary:  {
    "northeast": {
        "lat": -33.7263689,
        "lng": 150.9669451
    },
    "southwest": {
        "lat": -33.7583632,
        "lng": 150.9423336
    }
}
🌀-----------------------------------------------------------------🌀
Glenhaven
Glenhaven GPS : 151.00334 -33.70547
Boundary:  {
    "northeast": {
        "lat": -33.6886477,
        "lng": 151.0246956
    },
    "southwest": {
        "lat": -33.7167906,
        "lng": 150.9673601
    }
}
🌀-----------------------------------------------------------------🌀
West Pennant Hills
West Pennant Hills GPS : 151.04697 -33.74504
Boundary:  {
    "northeast": {
        "lat": -33.7318222,
        "lng": 151.0574352
    },
    "southwest": {
        "lat": -33.7634422,
        "lng": 151.0085662
    }
}
🌀-----------------------------------------------------------------🌀
Castle Hill - Central
Castle Hill - Central GPS : 151.0040814 -3

Seven Hills - Toongabbie
Seven Hills - Toongabbie GPS : 150.95645 -33.78606
Boundary:  {
    "northeast": {
        "lat": -33.7767608,
        "lng": 150.9675141
    },
    "southwest": {
        "lat": -33.8072276,
        "lng": 150.925868
    }
}
🌀-----------------------------------------------------------------🌀
Blacktown (South)
Blacktown (South) GPS : 150.9063 -33.771
Boundary:  {
    "northeast": {
        "lat": -33.747655,
        "lng": 150.9277733
    },
    "southwest": {
        "lat": -33.8046632,
        "lng": 150.8784362
    }
}
🌀-----------------------------------------------------------------🌀
Blacktown (West)
Blacktown (West) GPS : 150.8927967 -33.7704419
Boundary:  {
    "northeast": {
        "lat": -33.7690929197085,
        "lng": 150.8941456802915
    },
    "southwest": {
        "lat": -33.7717908802915,
        "lng": 150.8914477197085
    }
}
🌀-----------------------------------------------------------------🌀
Blacktown - North
Blacktown - North GPS : 150.9

...

Emu Plains - Leonay
Emu Plains - Leonay GPS : 150.64568 -33.76506
Boundary:  {
    "northeast": {
        "lat": -33.7565623,
        "lng": 150.6594457
    },
    "southwest": {
        "lat": -33.7804719,
        "lng": 150.6396296
    }
}
🌀-----------------------------------------------------------------🌀
Glenmore Park - Regentville
Glenmore Park - Regentville GPS : 150.6604462 -33.7711622
Boundary:  {
    "northeast": {
        "lat": -33.76434649999999,
        "lng": 150.6722845
    },
    "southwest": {
        "lat": -33.7783981,
        "lng": 150.6541015
    }
}
🌀-----------------------------------------------------------------🌀
Jamisontown - South Penrith
Jamisontown - South Penrith GPS : 150.7011034 -33.7715054
Boundary:  {
    "northeast": {
        "lat": -33.7613719,
        "lng": 150.7120459
    },
    "southwest": {
        "lat": -33.7855466,
        "lng": 150.6854557
    }
}
🌀-----------------------------------------------------------------🌀
Kingswood - Werrington


Regents Park
Regents Park GPS : 151.02833 -33.88572
Boundary:  {
    "northeast": {
        "lat": -33.8759795,
        "lng": 151.040129
    },
    "southwest": {
        "lat": -33.8887842,
        "lng": 151.0151883
    }
}
🌀-----------------------------------------------------------------🌀
Carlingford
Carlingford GPS : 151.048 -33.777
Boundary:  {
    "northeast": {
        "lat": -33.7584647,
        "lng": 151.068076
    },
    "southwest": {
        "lat": -33.7917183,
        "lng": 151.0212048
    }
}
🌀-----------------------------------------------------------------🌀
Carlingford
duplication !!! 🐳🐳🐳
Ermington - Rydalmere
Ermington - Rydalmere GPS : 151.06178 -33.81145000000001
Boundary:  {
    "northeast": {
        "lat": -33.7965837,
        "lng": 151.0704948
    },
    "southwest": {
        "lat": -33.8236603,
        "lng": 151.0504854
    }
}
🌀-----------------------------------------------------------------🌀
Oatlands - Dundas Valley
Oatlands - Dundas Valley GPS : 151.0

Pennant Hills - Cheltenham
Pennant Hills - Cheltenham GPS : 151.07569 -33.75839
Boundary:  {
    "northeast": {
        "lat": -33.7491017,
        "lng": 151.0888626
    },
    "southwest": {
        "lat": -33.7646052,
        "lng": 151.0654286
    }
}
🌀-----------------------------------------------------------------🌀
Ryde - Hunters Hill
Ryde - Hunters Hill GPS : 151.1384279 -33.8278214
Boundary:  {
    "northeast": {
        "lat": -33.8237825,
        "lng": 151.1439088
    },
    "southwest": {
        "lat": -33.833596,
        "lng": 151.1316369
    }
}
🌀-----------------------------------------------------------------🌀
Eastwood - Denistone
Eastwood - Denistone GPS : 151.08648 -33.80085
Boundary:  {
    "northeast": {
        "lat": -33.7928996,
        "lng": 151.1005117
    },
    "southwest": {
        "lat": -33.804494,
        "lng": 151.0791029
    }
}
🌀-----------------------------------------------------------------🌀
Gladesville - Huntleys Point
Gladesville - Huntleys 

Cabramatta - Lansvale
Cabramatta - Lansvale GPS : 150.9685549 -33.906913
Boundary:  {
    "northeast": {
        "lat": -33.8901408,
        "lng": 150.9795527
    },
    "southwest": {
        "lat": -33.9109701,
        "lng": 150.9431067
    }
}
🌀-----------------------------------------------------------------🌀
Cabramatta West - Mount Pritchard - Sydney
Cabramatta West - Mount Pritchard - Sydney - Sydney
Cabramatta West - Mount Pritchard - Sydney - Sydney - Sydney
Cabramatta West - Mount Pritchard - Sydney - Sydney - Sydney - Sydney
Cabramatta West - Mount Pritchard
Cabramatta West - Mount Pritchard GPS : 150.9060323 -33.8952273
Boundary:  {
    "northeast": {
        "lat": -33.8939224,
        "lng": 150.914916
    },
    "southwest": {
        "lat": -33.8979781,
        "lng": 150.8966942
    }
}
🌀-----------------------------------------------------------------🌀
😂-----------------------------------------------------------------😂
😂-----------------------------------------------

Cronulla - Kurnell - Bundeena
Cronulla - Kurnell - Bundeena GPS : 151.1515469 -34.08341619999999
Boundary:  {
    "northeast": {
        "lat": -34.0748052,
        "lng": 151.1605037
    },
    "southwest": {
        "lat": -34.0950227,
        "lng": 151.1405718
    }
}
🌀-----------------------------------------------------------------🌀
Lilli Pilli - Port Hacking - Dolans Bay
Lilli Pilli - Port Hacking - Dolans Bay GPS : 151.12641 -34.06259
Boundary:  {
    "northeast": {
        "lat": -34.0572414,
        "lng": 151.1325586
    },
    "southwest": {
        "lat": -34.0678081,
        "lng": 151.1228406
    }
}
🌀-----------------------------------------------------------------🌀
Woolaware - Burraneer
Woolaware - Burraneer GPS : 151.1398214 -34.055859
Boundary:  {
    "northeast": {
        "lat": -34.0392894,
        "lng": 151.145603
    },
    "southwest": {
        "lat": -34.0725089,
        "lng": 151.1348109
    }
}
🌀------------------------------------------------------------

In [15]:
# create 6 new column
data_StatisticalAreas['GPS_lng']=np.nan
data_StatisticalAreas['GPS_lat']=np.nan
data_StatisticalAreas['Location_northeast_lat']=np.nan
data_StatisticalAreas['Location_northeast_lng']=np.nan
data_StatisticalAreas['Location_southwest_lat']=np.nan
data_StatisticalAreas['Location_southwest_lng']=np.nan

# save GPS
count = 0
for j in data_StatisticalAreas['area_name']:
    for k,v in GPS.items():
        if j == k:
            data_StatisticalAreas['GPS_lng'][count] = v[0]
            data_StatisticalAreas['GPS_lat'][count] = v[1]
    count += 1
    
# save location
count = 0
for j in data_StatisticalAreas['area_name']:
    for k,v in Location.items():
        if j == k:
            s = ast.literal_eval(str(v[0].replace('\n','').replace(" ","").strip()))
            for k,v in s.items():
                if k == "northeast":
                    for k1, v1 in v.items():
                        if k1 == 'lat':
                            data_StatisticalAreas['Location_northeast_lat'][count]=v1
                        elif k1 == "lng":
                            data_StatisticalAreas['Location_northeast_lng'][count]=v1
                elif k == "southwest":
                    for k1, v1 in v.items():
                        if k1 == 'lat':
                            data_StatisticalAreas['Location_southwest_lat'][count]=v1
                        elif k1 == "lng":
                            data_StatisticalAreas['Location_southwest_lng'][count]=v1
            
    count += 1

In [16]:
data_StatisticalAreas.to_csv("new_StatisticalAreas.csv",index=False)

In [ ]:
#data_StatisticalAreas.info()

In [ ]:
data_StatisticalAreas.to_csv("new_StatisticalAreas.csv",index=False)